<a href="https://colab.research.google.com/github/Lior-Baruch/LLM-Advanced-FineTuning/blob/main/Gemma_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U transformers trl peft bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from transformers import TrainingArguments
from peft import LoraConfig, AutoPeftModelForCausalLM
from datasets import load_dataset, Dataset
from trl import SFTTrainer, DPOTrainer

from huggingface_hub import notebook_login
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")



Using cuda


In [5]:
from huggingface_hub import notebook_login

# log in to the Hugging Face hub (required for private datasets/models)
notebook_login()

In [3]:
def print_trainable_params(model):
    total_params = 0
    trainable_params  = 0
    for name, param in model.named_parameters():
        total_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || total params: {total_params} || trainable%: {100 * trainable_params / total_params}"
    )

In [6]:
model_name = "google/gemma-2b-it"
#model_name = "drive-download-alexStreetSmallModel"
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Set quantization config (to save memory)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(model_name, config=quantization_config)
model = model.to(device)
print_trainable_params(model)


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 2506172416 || total params: 2506172416 || trainable%: 100.0


In [7]:
# system = "You are a motivational interviewing counselor. You partner with the patient to understand his problems. You are empathetic towards him and help the patient explore their ambivalence regarding behavioral change. You are non-judgmental while encouraging the patient to change"

system_text = "<start_of_turn>system\n You are a motivational interviewing counselor. You partner with the patient to understand his problems. You are empathetic towards him and help the patient explore their ambivalence regarding behavioral change. You are non-judgmental while encouraging the patient to change.<end_of_turn>\n"
messages = [
        {"role": "user", "content": "I smoke a lot and I want to quit. I have tried to quit many times but I always fail. I don't know what to do."}
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print("chat template prompt: ", prompt)
#prompt = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)

prompt = system_text + prompt
print("with system text: ", prompt)
# transform prompt from string to tensor
prompt = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)
print("promt tensor: ", prompt)

# generate response
model.eval()
with torch.no_grad():
    response = model.generate(prompt, max_length=500, temperature=0.9, num_return_sequences=1, do_sample=True)

print(tokenizer.decode(response[0], skip_special_tokens=True))


chat template prompt:  <bos><start_of_turn>user
I smoke a lot and I want to quit. I have tried to quit many times but I always fail. I don't know what to do.<end_of_turn>
<start_of_turn>model

with system text:  <start_of_turn>system
 You are a motivational interviewing counselor. You partner with the patient to understand his problems. You are empathetic towards him and help the patient explore their ambivalence regarding behavioral change. You are non-judgmental while encouraging the patient to change.<end_of_turn>
<bos><start_of_turn>user
I smoke a lot and I want to quit. I have tried to quit many times but I always fail. I don't know what to do.<end_of_turn>
<start_of_turn>model

promt tensor:  tensor([[     2,    106,   9020,    108,   1646,    708,    476,  75297,  96839,
          70572, 235265,   1646,   9670,    675,    573,   7679,    577,   3508,
            926,   4552, 235265,   1646,    708, 200059,   6643,   1357,    578,
           1707,    573,   7679,  15370,   1024, 

prepare LoRA and model_ref

In [11]:
# load prompts_miti_6_train.json local dataset
dataset = load_dataset("json", data_files="prompts_miti_6_train.json") # DatasetDict with a single key: "train"
print("dataset keys: \n", dataset.keys()) # "train
print("\ndataset: \n", dataset) # DatasetInfo



train_dataset = dataset["train"]
df_train_data = dataset["train"].to_pandas()
print("\ntrain_dataset:\n ", train_dataset)
print("\ndf_train_data: \n", df_train_data.head())
sample = train_dataset[0]
print("\nsample keys: \n", sample.keys())
print("\nsample: \n", sample)
sample_msg = sample["messages"]
print("\nsample_msg: ", sample_msg)


dataset keys: 
 dict_keys(['train'])

dataset: 
 DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 5999
    })
})

train_dataset:
  Dataset({
    features: ['messages'],
    num_rows: 5999
})

df_train_data: 
                                             messages
0  [{'content': 'You are a motivational interview...
1  [{'content': 'You are a motivational interview...
2  [{'content': 'You are a motivational interview...
3  [{'content': 'You are a motivational interview...
4  [{'content': 'You are a motivational interview...

sample keys: 
 dict_keys(['messages'])

sample: 
 {'messages': [{'content': 'You are a motivational interviewing counselor. You partner with the patient to understand his problems. You are empathetic towards him and help the patient explore their ambivalence regarding behavioral change. You are non-judgmental while encouraging the patient to change', 'role': 'system'}, {'content': 'Um hmm.', 'role': 'user'}, {'content': "And so when

In [15]:
def formatting_func(example):
#    text = "<start_of_turn>system\n" + example["messages"][0]["content"] +"<end_of_turn>\n" + tokenizer.apply_chat_template(example["messages"][1:], tokenize=False, add_generation_prompt=True)
    text = tokenizer.apply_chat_template(example["messages"][1:], tokenize=False, add_generation_prompt=True)
    return text

#test formatting function
print(formatting_func(train_dataset[5]))


# Define LoRA ("low-rank attention") config
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

<bos><start_of_turn>user
no wonder I like them so much<end_of_turn>
<start_of_turn>model
all right have you heard of serotonin<end_of_turn>
<start_of_turn>user
yes I had<end_of_turn>
<start_of_turn>model
kind of a happy hormone but yeah so as we do eat simple carbohydrates in particular those raise our serotonin levels so that we actually do feel better and so some people on the Atkins diet actually do feel depressed and fatigued because their serotonin levels aren't as high but that can be a dangerous cycle because as we raise our blood sugar you know we feel good if it's a simple sugar though blood sugar goes up instantly goes up and then we tend to crash we tend to eat more which tends to do the cycle of carbohydrate cravings<end_of_turn>
<start_of_turn>user
so that's why I crave that bag of potato chips<end_of_turn>
<start_of_turn>model
I think I<end_of_turn>
<start_of_turn>model



In [16]:

from transformers import TrainingArguments

# Define training arguments

output_dir = "LBK95/gemma-2b-qlora-alexStreetSmall" # Set output directory for fine-tuned model
per_device_train_batch_size = 4  # Set batch size per device for training
gradient_accumulation_steps = 4  # Set the number of gradient accumulation steps
optim = "paged_adamw_32bit" # Set optimizer
save_steps = 10 # Set the number of steps before saving
logging_steps = 5 # Set the number of steps before logging
learning_rate = 2e-4 # Set learning rate
max_grad_norm = 0.3 # Set max gradient norm
max_steps = 200 # Set max steps
warmup_ratio = 0.03 # Set warmup ratio for learning rate scheduler
lr_scheduler_type = "cosine" # Set learning rate scheduler type


# Create TrainingArguments object with all of our defined arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)


model.train()
# Create SFTTrainer object (it is a wrapper around the Trainer object for Supervised-Fine-Tuning)
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    peft_config=lora_config,
    packing=True,
    tokenizer=tokenizer,
    max_seq_length=1024,
    formatting_func=formatting_func,
)



Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [17]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,6.655200
10,5.472800
15,4.559400
20,4.005000
25,3.703000
30,3.460800
35,3.201200
40,3.010600
45,2.875800
50,2.817400


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=200, training_loss=2.9175579881668092, metrics={'train_runtime': 2825.7448, 'train_samples_per_second': 1.132, 'train_steps_per_second': 0.071, 'total_flos': 3.896471461665178e+16, 'train_loss': 2.9175579881668092, 'epoch': 2.55})

In [18]:
# Save the model to the hub
trainer.model.save_pretrained(output_dir)

In [37]:
# system = "You are a motivational interviewing counselor. You partner with the patient to understand his problems. You are empathetic towards him and help the patient explore their ambivalence regarding behavioral change. You are non-judgmental while encouraging the patient to change"

#system_text = "<start_of_turn>system\n You are a motivational interviewing counselor. You partner with the patient to understand his problems. You are empathetic towards him and help the patient explore their ambivalence regarding behavioral change. You are non-judgmental while encouraging the patient to change.<end_of_turn>\n"
messages = [
        {"role": "user", "content": "I smoke a lot and I want to quit. I have tried to quit many times but I always fail. I don't know what to do."},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print("chat template prompt: ", prompt)


#
# transform prompt from string to tensor
prompt = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)
print("promt tensor: ", prompt)

# generate response
model.eval()
with torch.no_grad():
    response = model.generate(prompt, max_length=100, temperature=0.9, num_return_sequences=1, do_sample=True)

print(tokenizer.decode(response[0], skip_special_tokens=True))




chat template prompt:  <bos><start_of_turn>user
I smoke a lot and I want to quit. I have tried to quit many times but I always fail. I don't know what to do.<end_of_turn>
<start_of_turn>model

promt tensor:  tensor([[     2,      2,    106,   1645,    108, 235285,  15627,    476,   2940,
            578,    590,   1938,    577,  16640, 235265,    590,    791,   6423,
            577,  16640,   1767,   3023,    901,    590,   2593,   5967, 235265,
            590,   1453, 235303, 235251,   1230,   1212,    577,    749, 235265,
            107,    108,    106,   2516,    108]], device='cuda:0')
user
I smoke a lot and I want to quit. I have tried to quit many times but I always fail. I don't know what to do.
model
It would be a good idea to speak with a doctor or other health professional about smoking. They can help you with a smoking cessation plan that is right for you. Would you like to contact your doctor or another health professional to arrange an appointment?
user
Yes, yes.


In [42]:
# push to hub
trainer.push_to_hub()

# load from hub to test
loaded_model = AutoModelForCausalLM.from_pretrained(output_dir)


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

events.out.tfevents.1709457222.bda87d3e2094.1176.0:   0%|          | 0.00/13.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/LBK95/gemma-2b-qlora-alexStreetSmall/commit/44f4a16ff8743b63e3b3f64105a73f48a2213833', commit_message='End of training', commit_description='', oid='44f4a16ff8743b63e3b3f64105a73f48a2213833', pr_url=None, pr_revision=None, pr_num=None)

In [46]:
SFT_model_id = "LBK95/gemma-2b-qlora-alexStreetSmall"
base_model_id = "google/gemma-2b-it"
# load model from hub
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

loaded_model = AutoPeftModelForCausalLM.from_pretrained(
    SFT_model_id, # location of saved SFT model
    low_cpu_mem_usage=True,
    is_trainable=False,
    device_map="auto",
    quantization_config=quantization_config,
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [55]:
# system = "You are a motivational interviewing counselor. You partner with the patient to understand his problems. You are empathetic towards him and help the patient explore their ambivalence regarding behavioral change. You are non-judgmental while encouraging the patient to change"

#system_text = "<start_of_turn>system\n You are a motivational interviewing counselor. You partner with the patient to understand his problems. You are empathetic towards him and help the patient explore their ambivalence regarding behavioral change. You are non-judgmental while encouraging the patient to change.<end_of_turn>\n"
messages = [
        {"role": "user", "content": "I smoke a lot and I want to quit."},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print("chat template prompt: ", prompt)


#
# transform prompt from string to tensor
prompt = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)
print("promt tensor: ", prompt)

# generate response
#loaded_model.to(device)
loaded_model.eval()
with torch.no_grad():
    response = loaded_model.generate(prompt, max_length=250, temperature=0.9, num_return_sequences=1, do_sample=True)

print("SFT model:")
print(tokenizer.decode(response[0], skip_special_tokens=True))


# generate response
base_model.eval()
with torch.no_grad():
    response = base_model.generate(prompt, max_length=250, temperature=0.9, num_return_sequences=1, do_sample=True)

print("\nBase model:")
print(tokenizer.decode(response[0], skip_special_tokens=True))

chat template prompt:  <bos><start_of_turn>user
I smoke a lot and I want to quit.<end_of_turn>
<start_of_turn>model

promt tensor:  tensor([[     2,      2,    106,   1645,    108, 235285,  15627,    476,   2940,
            578,    590,   1938,    577,  16640, 235265,    107,    108,    106,
           2516,    108]], device='cuda:0')
SFT model:
user
I smoke a lot and I want to quit.
model
It's great that you're thinking about doing something to stop smoking, I know it can be tough, but having quit smoking is one of the things that matters most to be able to save your health. There is a lot of information out there and that when you're thinking about quitting you want to keep that information up front. You're going to have to do some things while you're smoking on your own and you have some choices to make as well. You have to know that you have options and that you're not just going to quit smoking right away like your friends and family were going to be excited and excited. Have you